# Area statistics for protected areas in mainland Norway

[![colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ac-willeke/pygdal-geo-engineer/blob/main/notebooks/2024.01_conservation_and_preservation.ipynb) [![github](https://img.shields.io/badge/GitHub-View%20on%20GitHub-blue?logo=github)](https://github.com/ac-willeke/pygdal-geo-engineer/blob/main/notebooks/2024.01_conservation_and_preservation.ipynb)

**Author**: Willeke A'Campo

**Description:** This notebook calculates area statistics for protected areas in mainland Norway. Area statistics are calculated for the datasetes **Verneområder** og **Foreslatte verneområder** from the Norwegian Environment Agency. 

The area statistics can be divided in three groups:

1. Area variables for the protected areas:
    - Area (terrestrial and marine)
    - Perimeter (terrestrial and marine)
    - Land area (terrestrial)
    - Perimeter land area (terrestrial)

2. Overlay statistics for the protected area and the following datasets:
    - AR50 - Bonitet
        - Area proportion of land cover quality classes 
    - Bioklimatiske soner
        - Area proportion of bioclimatic zone classes
    - Infrastrukturindeks: 
        - Mean value of infrastructure index per protected area 
        - Area proportion of infrastructure index classes
    - Høydelag
        - Area proportion of elevation classes 

3. Spatial indices for the protected areas:
    - Density of protected area per 10x10km (SSB grid)
    - Shape Index for each protected area (land + marine)
    - Shape index for each protected area (land)






## Configure Workflow

In [1]:
# Variables to control the execution of the workflow
gis_server = True # True = use GIS server, False = use local files
load_data = False # True = load data from source, False = data is already loaded
test_area = False # True = test area, False = full area
prepare_duckdb = False # True = prepare duckdb, False = duckdb is already prepared
run_ar50 = False # True = run AR50 area calc , False = AR50 is already run

In [2]:
import os 
from osgeo import gdal
from pathlib import Path
from itertools import islice

from shapely.geometry import box
import pandas as pd
import geopandas as gpd

# mapping libraries
# https://leafmap.org/faq/#how-to-use-a-specific-plotting-backend
import leafmap.foliumap as leafmap

In [3]:
project_path= Path.cwd().parents[0]
shell_path= os.path.join(project_path, "src", "shell")
python_path= os.path.join(project_path, "src", "python")

if gis_server:
    user = "willeke.acampo" #input("Username: firstname.lastname")
    data_path = f"/home/NINA.NO/{user}/Mounts/scratch/wilaca/vern_og_bevaring/data"
else:
    data_path = os.path.join(project_path, "data")

print(f"Gdal Version: {gdal.__version__} ")
print(f"Project Path: {project_path}")
print(f"Path to shell scripts: {shell_path}")
print(f"Path to python scripts: {python_path}")
print(f"Path to data: {data_path}")

Gdal Version: 3.8.3 
Project Path: /home/NINA.NO/willeke.acampo/git/pygdal-geo-engineer
Path to shell scripts: /home/NINA.NO/willeke.acampo/git/pygdal-geo-engineer/src/shell
Path to python scripts: /home/NINA.NO/willeke.acampo/git/pygdal-geo-engineer/src/python
Path to data: /home/NINA.NO/willeke.acampo/Mounts/scratch/wilaca/vern_og_bevaring/data


In [4]:
# import local scripts
import sys 
sys.path.append(python_path)

from ogr_utils import import_gpkg, print_layer_schema
from lookup import create_lookup_dict, lookup_value
from duckdb_utils import (load_gpkg_layers, blob_to_geom, remove_field, \
    ar50_area_class, sum_area_cols, geom_area, geom_peri, area_difference)

## Download and Import Data

Get the data and import into geopackages `vern_25833.gpkg` and `bevaring_25833.gpkg` using ogr2ogr.

```python
# change path to data folder
os.chdir(os.path.join(data_path, "raw"))
print(f"Download data to: {os.getcwd()}")
```

```bash
# ArcGIS REST: Naturvern (02.2024)
ogr2ogr -f "GPKG" -t_srs EPSG:25833 -nln naturvernomrade -nlt MULTIPOLYGON vern_25833.gpkg "https://kart.miljodirektoratet.no/arcgis/rest/services/vern/mapserver/0/query?where=1%3D1&outfields=*&f=json&resultRecordCount=1"

# ArcGIS REST: Foreslåtte verneområder (02.2024)
ogr2ogr -f "GPKG" -t_srs EPSG:25833 -nln foreslatt_vern -nlt MULTIPOLYGON vern_25833.gpkg "https://kart.miljodirektoratet.no/arcgis/rest/services/vern/mapserver/4/query?where=1%3D1&outfields=*&f=json&resultRecordCount=1"

# AR50 - 2022 
# Stored on NINA's server /GeoSpatialData

# Bioklima
# Received from V. Bakkestuen (NINA)

# SSB grid
curl -o Ruter_10KM_norge.zip "https://www.ssb.no/natur-og-miljo/_attachment/375082?_ts=1685c0e69b8"
unzip Ruter_10KM_norge.zip

# Fylkerområder
# Stored on NINA's server /GeoSpatialData

# Infrastruktur
# Received from V. Bakkestuen (NINA)
```

Downloaded datasets are stored in the `/data` folder:

**vern_25833.gpkg** contains the following layers:
| Layer | Dataset Name | Description | Year | Source |
|-------| ------------ | ----------- | ---- | ------ |
| naturvernomrade | Naturvernområder| Nature protected areas | 2024 | [Miljødirektoratet](https://kartkatalog.miljodirektoratet.no/Dataset/Details/0) |
| foreslatt_vern | Foreslåtte naturvernområder | Planned nature protected areas | 2024 | [Miljødirektoratet](https://kartkatalog.miljodirektoratet.no/Dataset/Details/1)| 

**bevaring_25833.gpkg** contains the following layers:
|Layer| Dataset Name | Description | Year | Source |
|-----| ------------ | ----------- | ---- | ------ |
| ar50_2022 | AR50 | Land cover classes | 2022 | [NIBIO](https://kart8.nibio.no/nedlasting/dashboard) |
| bioklima_soner_2017 | Bioklimatiske soner | Bioclimatic zones | 2017 | [Artsdatabanken](https://data.artsdatabanken.no/Natur_i_Norge/Natursystem/Beskrivelsessystem/Regional_naturvariasjon/Bioklimatisk_sone) |

**other files**

| Filename | Dataset Name | Description | Year | Source |
|----------| ------------ | ----------- | ---- | ------ |
| ssb_grid_5km.geojson | SSB rutenett (5x5 km)| Grid for Norway | 2024 | [SSB](https://kart.ssb.no/) |
| fylker_2024.geojson | Fylker, 2024 | Provincial Boundaries | 2024 | [SSB](https://www.ssb.no/en/kart/griddata) |
| Infra25m.tif | Infrastrukturindeks | Infrastructure index (25m)| 2024 | Internal NINA datasett |

In [5]:
if load_data: 
    # run shell scripts from /src/shell
    os.chdir(shell_path)

    input_file = "vern_25833.gpkg"
    path = os.path.join(data_path, "interim")

    ! gdalinfo --version
    ! chmod +x gdal_gpkg-info.sh
    ! ./gdal_gpkg-info.sh {path} {input_file}

## Data Preparation

### Sudy Area | Norway Mainland

This workflow is created and run for Norway mainland. If you like to test or use the workflow we recommend running it first on a study area. Below are bounding box coordinates provided for Dovrefjell, Fosen or Trondheim area. 

**Create Bounding Box** 

In [6]:
# bounding box Dovrefjell
bounding_box = "fosen"  # "dovrefjell" or "trondheim"

if bounding_box == "dovrefjell":
    xmin, ymin, xmax, ymax = 160000.00, 6900000.00, 260000.00, 6950000.00
if bounding_box == "fosen":
    xmin, ymin, xmax, ymax = 180000.00, 7010000.00, 290000.00, 7150000.00
if bounding_box == "trondheim":
    xmin, ymin, xmax, ymax = 260520.12, 7032142.5, 278587.56, 7045245.27

# Create a bounding box
boxBB =  box(xmin, ymin, xmax, ymax)
crs = 'EPSG:25833'

gdf_BB = gpd.GeoDataFrame(geometry=[boxBB])
gdf_BB['name'] = 'Dovre_BB'
gdf_BB.crs = crs
bounds = gdf_BB.bounds.to_numpy().tolist()[0]

In [7]:
if load_data: 
    # run shell scripts from /src/shell
    os.chdir(shell_path)
    for file in ["vern", "bevaring", "admin"]:
        input_file = os.path.join(data_path, "interim", f"{file}_25833.gpkg")
        output_file = os.path.join(data_path, "tmp", f"{file}_25833_bbox.gpkg")
        
        #for layer_name in layer_names:
        ! chmod +x gdal_copy-file-bbox.sh
        ! ./gdal_copy-file-bbox.sh {input_file} {output_file} {xmin} {ymin} {xmax} {ymax}

### AR50 - Bonitet

Translate the Bonitet classes using a lookup table.
 
&emsp; $Bonitet = (artype, artreslag, arskogbon, arjordbr, ardyrkning, arveget)$ 

In [8]:
if load_data: 
    # create a temporary test gpkg
    tmp_gpkg = os.path.join(data_path, "tmp", "tmp.gpkg")
    in_gpkg = os.path.join(data_path, "interim", "bevaring_25833.gpkg")
    print(tmp_gpkg)

    sql_query = f'SELECT * FROM "ar50_flate" LIMIT 5'
    path = os.path.join(data_path, "tmp")
    ! ogr2ogr -f "GPKG" -nln "ar50_flate" -sql "$sql_query" {tmp_gpkg} {in_gpkg}

In [9]:
# ar50 layer
layer_name, new_field_name = "ar50_flate", "ar50_bonitet"

if test_area:
    # import gpkg into ogr object 
    # add field name if it does not exist
    tmp_gpkg = os.path.join(data_path, "tmp", "bevaring_25833_bbox.gpkg")
    ds, lyr = import_gpkg(tmp_gpkg, layer_name, new_field_name)
    #print_layer_schema(lyr)
else:
    # import gpkg into ogr object 
    # add field name if it does not exist
    in_gpkg = os.path.join(data_path, "interim", "bevaring_25833.gpkg")
    ds, lyr = import_gpkg(in_gpkg, layer_name, new_field_name)
    #print_layer_schema(lyr)

# Convert the first 5 features of the layer to a DataFrame
# use islice to limit the number of features to 5, to reduce computation time
df_AR50 = pd.DataFrame(feature.items() for feature in islice(lyr, 5))

# Print the DataFrame
df_AR50

,OBJECTID,objtype,artype,arskogbon,artreslag,arveget,arjordbr,ardyrking,arkartstd,målemetode,informasjon,oppdateringsdato,originaldatavert,navnerom,kopidato,områdeid,lokalid,fylkeid,komid,ar50_bonitet
0,1,arealressursflate,60,13,31,98,98,81,AR50,64,Generalisert fra AR5 Årsversjon 2016,20180106,NIBIO,NO_NIBIO_AR50_2019_2,20211021,5022,1,50,5022,11
1,2,arealressursflate,60,11,31,98,98,81,AR50,64,Generalisert fra AR5 Årsversjon 2016,20180106,NIBIO,NO_NIBIO_AR50_2019_2,20211021,5022,2,50,5022,11
2,3,arealressursflate,60,13,31,98,98,81,AR50,64,Generalisert fra AR5 Årsversjon 2016,20180106,NIBIO,NO_NIBIO_AR50_2019_2,20211021,5022,3,50,5022,11
3,4,arealressursflate,60,11,31,98,98,81,AR50,64,Generalisert fra AR5 Årsversjon 2016,20180106,NIBIO,NO_NIBIO_AR50_2019_2,20211021,5022,4,50,5022,11
4,5,arealressursflate,30,13,31,98,98,81,AR50,64,Generalisert fra AR5 Årsversjon 2016,20180106,NIBIO,NO_NIBIO_AR50_2019_2,20211021,5022,5,50,5022,4


In [10]:
# lookup table 
lookup_csv = os.path.join(data_path, "AR50_bonitet_lookup.csv")
lookup_df = pd.read_csv(lookup_csv)

# rename cols to correspond with AR50 lyr
lookup_df.rename(
    columns={
        "ARTYPE kode": "artype",
        "ARTRESLAG kode": "artreslag",
        "ARSKOGBON kode": "arskogbon",
        "ARJORDBR kode": "arjordbr",
        "ARDYRKING kode": "ardyrking",
        "ARVEGET kode": "arveget",
        "Bonitet kode": "ar50_bonitet",
    },
    inplace=True,
)

# reorder cols to correspond with AR50 lyr
lookup_df = lookup_df[
    [
        "artype",
        "arskogbon",
        "artreslag",
        "arveget",
        "arjordbr",
        "ardyrking",
        "Beskrivelse",
        "ar50_bonitet",
    ]
]

display(lookup_df.head(11))

,artype,arskogbon,artreslag,arveget,arjordbr,ardyrking,Beskrivelse,ar50_bonitet
0,10,98,98,98,98,98,Bebygd og samferdsel,14
1,20,98,98,98,24,98,Fulldyrka og Overflatedyrka jord,1
2,20,98,98,98,25,82,"Innmarksbeite, dyrkbar jord",2
3,20,98,98,98,25,81,Innmarksbeite,2
4,20,98,98,98,99,99,"Jordbruksareal, uspesifisert",1
5,30,11,31,98,98,82,"Barskog, impediment, dyrkbar jord",6
6,30,11,31,98,98,81,"Barskog, impediment",6
7,30,12,31,98,98,82,"Barskog, lav skogbonitet, dyrkbar jord",5
8,30,12,31,98,98,81,"Barskog, lav skogbonitet",5
9,30,13,31,98,98,82,"Barskog, middels skogbonitet, dyrkbar jord",4


In [11]:
# create lookup dict
# keys must be in same order as gpkg lyr fields
keys = (
    "artype",
    "arskogbon",
    "artreslag",
    "arveget",
    "arjordbr",
    "ardyrking",
    )

value= "ar50_bonitet"

lookup_dict = create_lookup_dict(
    lookup_df,
    keys=keys,
    value=value
)

# print first two entries of dict
print({k: lookup_dict[k] for k in list(lookup_dict)[:11]})

{(10, 98, 98, 98, 98, 98): 14, (20, 98, 98, 98, 24, 98): 1, (20, 98, 98, 98, 25, 82): 2, (20, 98, 98, 98, 25, 81): 2, (20, 98, 98, 98, 99, 99): 1, (30, 11, 31, 98, 98, 82): 6, (30, 11, 31, 98, 98, 81): 6, (30, 12, 31, 98, 98, 82): 5, (30, 12, 31, 98, 98, 81): 5, (30, 13, 31, 98, 98, 82): 4, (30, 13, 31, 98, 98, 81): 4}


In [12]:
if load_data:
    # loop through the features and reclassify the attribute value "ar50_bonitet"
    features_to_update = []

    for feature in lyr:
        # get the attribute values
        artype = feature.GetField("artype")
        arskogbon = feature.GetField("arskogbon")
        artreslag = feature.GetField("artreslag")
        arveget = feature.GetField("arveget")
        arjordbr = feature.GetField("arjordbr")
        ardyrking = feature.GetField("ardyrking")

        key = (int(artype), int(arskogbon), int(artreslag), int(arveget), int(arjordbr), int(ardyrking))
        if key in lookup_dict:
            new_value = lookup_dict[key]
            feature.SetField("ar50_bonitet", new_value)
            features_to_update.append(feature)

    print(f"Number of features to update: {len(features_to_update)}")
    # Batch update features
    for feature in features_to_update:
        lyr.SetFeature(feature)

    print("Finished updating Bonitet.")

    df_AR50_bon = pd.DataFrame(feature.items() for feature in islice(lyr, 5))
    display(df_AR50_bon.head())

# close OGR object
ds = None

In [13]:
if load_data:
    # Define the SQL queries
    sql_marine = f"SELECT * FROM ar50_flate WHERE ar50_bonitet = 17"
    sql_terrestisk = f"SELECT * FROM ar50_flate WHERE ar50_bonitet != 17"

    if test_area:
        # import gpkg into ogr object 
        # add field name if it does not exist
        in_gpkg = os.path.join(data_path, "tmp", "bevaring_25833_bbox.gpkg")
        out_gpkg = os.path.join(data_path, "tmp", "interim_25833_bbox.gpkg")
    else:
        # import gpkg into ogr object 
        # add field name if it does not exist
        in_gpkg = os.path.join(data_path, "interim", "bevaring_25833.gpkg")
        out_gpkg = os.path.join(data_path, "interim", "interim_25833.gpkg")

    print(out_gpkg)

    # Create the ar50_marine layer
    ! ogr2ogr -f "GPKG" -nln "ar50_marine" -sql "$sql_marine" -append {out_gpkg} {in_gpkg}
    # Create the ar50_terrestisk layer
    ! ogr2ogr -f "GPKG" -nln "ar50_terrestisk" -sql "$sql_terrestisk" -append {out_gpkg} {in_gpkg}

### Infrastructure index

- Calculate and display the distribution of the infrastructure index for the whole of norway 
- Calculate and display the distribution of the infrastructure index per region
- Calculate and display the distribution of the infrastructure index per protected area
- Define the infrastructure index classes


### Topographic height

Classify the topographic height into 4 classes:
- 0-300m
- 301-600m
- 601-900m
- over 900m

## Load data into Spatial Database (DuckDB)


### Load data into DuckDB

In [14]:
if test_area:
    db_path = os.path.join(data_path, "tmp", "verg_og_bevaring_tmp.db")
else:
    db_path = os.path.join(data_path, "interim", "verg_og_bevaring.db")

**Load AR50 into DuckDB**

In [15]:
if prepare_duckdb:
    # AR50 into duckdb
    if test_area:
        in_gpkg = os.path.join(data_path, "tmp", "bevaring_25833_bbox.gpkg")
        db_path = os.path.join(data_path, "tmp", "verg_og_bevaring_tmp.db")
        load_gpkg_layers(db_path, in_gpkg, "ar50_flate")
        load_gpkg_layers(db_path, in_gpkg, "ar50_flate_msk")
    else:
        in_gpkg = os.path.join(data_path, "interim", "bevaring_25833.gpkg")
        db_path = os.path.join(data_path, "interim", "verg_og_bevaring.db")
        load_gpkg_layers(db_path, in_gpkg, "ar50_flate")

**Load Protected Areas into DuckDB**

In [16]:
# import protected areas into DuckDB

# if Test is True, import test gpkg otherwise import the original gpkg
if prepare_duckdb:
    if test_area:
        in_gpkg = os.path.join(data_path, "tmp", "vern_25833_bbox.gpkg")
        db_path = os.path.join(data_path, "tmp", "verg_og_bevaring_tmp.db")
        load_gpkg_layers(db_path, in_gpkg, "naturvernomrade")
        load_gpkg_layers(db_path, in_gpkg, "foreslatt_vern")
    else:
        in_gpkg = os.path.join(data_path, "interim", "vern_25833.gpkg")
        db_path = os.path.join(data_path, "interim", "verg_og_bevaring.db")
        load_gpkg_layers(db_path, in_gpkg, "naturvernomrade")
        load_gpkg_layers(db_path, in_gpkg, "foreslatt_vern")

### Preprocess DuckDB Tables 

**Convert BLOB columns to GEOM columns**

In [17]:
if prepare_duckdb:
    # cast BLOB (Binary Large OBject) to geometry for spatial operations
    
    import duckdb

    con = duckdb.connect(db_path)
    con.install_extension('spatial')
    con.load_extension('spatial')
    
    # duckdb tables names to list
    tables = con.execute("SHOW TABLES;").fetchdf()
    tables = tables["name"].to_list()
    print(tables)
    
    for table in tables:
        
        # TODO add check if geom field exists
        
        blob_to_geom(
            db_path=db_path,
            tbl_name=table,
            blob_field="geometry",
            geom_field="geom",   
        )

### Display DuckDB tables 

In [18]:
import duckdb

con = duckdb.connect(db_path)
con.install_extension('spatial')
con.load_extension('spatial')

In [19]:
# print tables
con.sql("SHOW TABLES;")

┌─────────────────┐
│      name       │
│     varchar     │
├─────────────────┤
│ ar50_flate      │
│ foreslatt_vern  │
│ naturvernomrade │
└─────────────────┘

In [20]:
# display first 5 rows of the table
table = "foreslatt_vern"
con.sql(f"SELECT * FROM {table} LIMIT 5")

┌──────────────────────┬──────────────────────┬────────────┬───┬─────────────────┬────────────┬──────────────┐
│ identifikasjon_lok…  │ identifikasjon_nav…  │    navn    │ … │ sum_ar50_bon_m2 │  area_m2   │ area_diff_m2 │
│       varchar        │       varchar        │  varchar   │   │      float      │   float    │    float     │
├──────────────────────┼──────────────────────┼────────────┼───┼─────────────────┼────────────┼──────────────┤
│ VP00000313           │ http://www.miljodi…  │ Inngripan  │ … │       4227805.0 │  4227805.0 │          0.0 │
│ VP00000404           │ http://www.miljodi…  │ Kverndalen │ … │      10999388.0 │ 10999387.0 │          1.0 │
│ VP00000410           │ http://www.miljodi…  │ Båsdalen   │ … │       1617838.9 │  1617839.0 │       -0.125 │
│ VP00000416           │ http://www.miljodi…  │ Rennen     │ … │       1811234.4 │  1811234.5 │       -0.125 │
│ VP00000418           │ http://www.miljodi…  │ Leksa      │ … │       6441098.0 │  6441097.5 │          0.5 │
├

In [21]:
# close
con.close()

## Display Data on the Map

https://ipyleaflet.readthedocs.io/en/latest/installation/index.html#using-pip 


In [22]:
# Display data on map using leafmap
# TODO display duckdb tables as well, not only WMS

init_location = [62.223207, 9.550195]  # Hjerkinn
zoom_start = 13  
min_zoom = 8  # keeps user from zooming out too far
basemap = leafmap.Map(
    location=init_location, 
    zoom=zoom_start, 
    min_zoom=min_zoom, 
    max_bounds=True
    )

# set background
basemap.add_basemap("SATELLITE", opacity=0.7)

# add WMS verneområder
wms_url ="https://kart.miljodirektoratet.no/arcgis/services/vern/mapserver/WMSServer"
wms_layer = "naturvern_klasser_omrade"
wms_name = "Protected Areas"
basemap.add_wms_layer(
    url=wms_url, 
    layers=wms_layer, 
    name=wms_name, 
    wms_format="image/png",
    )

if test_area:
    # add bounding box - Hjerkinn
    basemap.add_gdf(gdf_BB, layer_name="Test Area", fill_color="blue", fill_opacity=0.2, weight=2)

basemap

## Methods

## 1. Overlap protected areas and AR50 (land cover)

**Divide the protected areas into terrestrial and marine areas**

AR50 polygons with classified as "Hav" are considered marine areas, all other areas are considered terrestrial.
- Bonitet = 17 (Hav) 
- Arealtype = 82 (Hav)

<br>

**AR50 - Bonitet**
Calculated area overlap with AR50 Bonitet classes. 

| Protected area | Fulldyrka og overflatedyrka jord (m2) | Innmarksbeite (m2) | Skog, høg og særs høg bonitet (m2) | ... |
|----------------|--------------------------------------|-------------------|-----------------------------------|-----|
| *NaturvernId*  |*AR50_bon_1*|*AR50_bon2*|*AR50_bon3*|...|
| area A         |                                      |                   |                                   |     |


In [23]:

tbl_study_area = "foreslatt_vern"
id = "identifikasjon_lokalId"
tbl_ar50 = "ar50_flate"
ar50_field = "ar50_bonitet"

if run_ar50:
    for area_class in range(1, 19): # 1-18
        print(f"Calculating Bonitet class {area_class}")
        new_field = f"ar50_bon{area_class}" # ar50_bon1
        remove_field(db_path, tbl_study_area, new_field)
        ar50_area_class(db_path, tbl_study_area, id, tbl_ar50, ar50_field, area_class, new_field)


In [24]:
# print first 5 rows of the table "foreslatt_vern"
con = duckdb.connect(db_path)
con.sql(f"SELECT * FROM foreslatt_vern LIMIT 5")

# print as dataframe
df = con.execute(f"SELECT * FROM foreslatt_vern LIMIT 5").fetchdf()

# display columns: identfifikasjon_lokalId, ar50_bon17
col_id= [id]
cols_ar50 = [col for col in df.columns if "ar50_bon" in col]
cols = col_id + cols_ar50
df_ar50 = df[cols]
display(df_ar50)

# close
con.close()


,identifikasjon_lokalId,ar50_bon1,ar50_bon2,ar50_bon3,ar50_bon4,ar50_bon5,ar50_bon6,ar50_bon7,ar50_bon8,ar50_bon9,ar50_bon10,ar50_bon11,ar50_bon12,ar50_bon13,ar50_bon14,ar50_bon15,ar50_bon16,ar50_bon17,ar50_bon18,sum_ar50_bon_m2
0,VP00000313,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.788251e+04,NaN,32083.675781,NaN,NaN,NaN,NaN,NaN,NaN,4117838.75,NaN,4.227805e+06
1,VP00000404,NaN,NaN,NaN,233783.78125,1.501154e+06,2.959454e+06,1400501.375,2.212473e+06,2.108428e+06,128523.148438,188387.609375,225231.843750,NaN,NaN,NaN,41450.535156,NaN,NaN,1.099939e+07
2,VP00000410,NaN,NaN,194796.46875,550123.81250,3.330043e+05,3.728594e+05,NaN,7.141531e+03,1.802707e+04,NaN,69037.406250,40107.097656,NaN,NaN,NaN,32741.802734,NaN,NaN,1.617839e+06
3,VP00000416,NaN,NaN,NaN,157918.78125,1.017872e+06,6.664241e+04,NaN,NaN,NaN,NaN,55577.246094,28114.167969,NaN,NaN,NaN,485110.375000,NaN,NaN,1.811234e+06
4,VP00000418,NaN,38501.339844,NaN,374454.43750,1.415678e+06,3.024922e+06,NaN,2.814419e+05,1.806216e+05,NaN,627368.187500,498110.281250,NaN,NaN,NaN,NaN,NaN,NaN,6.441098e+06


In [25]:
# Calculating sum of all bonitet classes
remove_field(db_path, tbl_study_area, "sum_ar50_bon_m2")
sum_area_cols(db_path, tbl_study_area, id, cols_ar50, "sum_ar50_bon_m2")

# Calculating area of each polygon in the study area
remove_field(db_path, tbl_study_area, "area_m2")
geom_area(db_path, tbl_study_area, "geom", "area_m2")

# Calculateing the difference between the sum of all bonitet classes and the area of each polygon
remove_field(db_path, tbl_study_area, "area_diff_m2")
area_difference(db_path, tbl_study_area, "sum_ar50_bon_m2", "area_m2", "area_diff_m2")


An error occurred: Binder Error: Referenced column "sum_ar50_bon_m2" not found in FROM clause!
Candidate bindings: "foreslatt_vern.ar50_bon2"
LINE 5: ...ar50_bon5, ar50_bon6, ar50_bon7, ar50_bon8, ar50_bon9, ar50_bon10, ar50_bon11, ar50_bon12, ar50_bon13, ar50_bon14, ar50_bon15, ar50_bon16, ar50_bon17, ar50_bon18, sum_ar50_bon_m2,
                                                  ^
An error occurred: Binder Error: Referenced column "sum_ar50_bon_m2" not found in FROM clause!
Candidate bindings: "study_area.ar50_bon2"
LINE 3: ...            UPDATE foreslatt_vern as study_area
                SET area_diff_...
                                                  ^


In [26]:

# print first 5 rows of the table "foreslatt_vern"
con = duckdb.connect(db_path)
con.sql(f"SELECT * FROM foreslatt_vern")

# print as dataframe
df = con.execute(f"SELECT * FROM foreslatt_vern").fetchdf()

# sort df by area_diff_m2
df_2 = df.sort_values(by="area_diff_m2", ascending=False)
display(df_2)

# close
con.close()

,identifikasjon_lokalId,identifikasjon_navnerom,navn,datafangstdato,faktaark,kommune,kvalitet_målemetode,kvalitet_nøyaktighet,verneform,verneplan,...,ar50_bon11,ar50_bon12,ar50_bon13,ar50_bon14,ar50_bon15,ar50_bon16,ar50_bon17,ar50_bon18,area_m2,area_diff_m2
0,VP00000313,http://www.miljodirektoratet.no/naturbase,Inngripan,20031217000000,https://faktaark.naturbase.no/?id=VP00000313,1505,NaN,NaN,naturreservat,,...,NaN,NaN,NaN,NaN,NaN,NaN,4117838.75,NaN,4.227805e+06,NaN
1,VP00000404,http://www.miljodirektoratet.no/naturbase,Kverndalen,20000101000000,https://faktaark.naturbase.no/?id=VP00000404,5038,NaN,NaN,naturreservat,,...,188387.609375,225231.843750,NaN,NaN,NaN,41450.535156,NaN,NaN,1.099939e+07,NaN
2,VP00000410,http://www.miljodirektoratet.no/naturbase,Båsdalen,20000101000000,https://faktaark.naturbase.no/?id=VP00000410,5042,NaN,NaN,naturreservat,,...,69037.406250,40107.097656,NaN,NaN,NaN,32741.802734,NaN,NaN,1.617839e+06,NaN
3,VP00000416,http://www.miljodirektoratet.no/naturbase,Rennen,20000101000000,https://faktaark.naturbase.no/?id=VP00000416,5035,NaN,NaN,naturreservat,,...,55577.246094,28114.167969,NaN,NaN,NaN,485110.375000,NaN,NaN,1.811234e+06,NaN
4,VP00000418,http://www.miljodirektoratet.no/naturbase,Leksa,20000101000000,https://faktaark.naturbase.no/?id=VP00000418,5035,NaN,NaN,naturreservat,,...,627368.187500,498110.281250,NaN,NaN,NaN,NaN,NaN,NaN,6.441098e+06,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284,VP00001394,http://www.miljodirektoratet.no/naturbase,Vinstradalen - utvidelse,20240115000000,https://faktaark.naturbase.no/?id=VP00001394,3436,NaN,NaN,naturreservat,skogvern,...,NaN,NaN,NaN,NaN,NaN,31963.154297,NaN,NaN,6.971127e+05,NaN
285,VP00001395,http://www.miljodirektoratet.no/naturbase,Sjoa - utvidelse,20240115000000,https://faktaark.naturbase.no/?id=VP00001395,3437,NaN,NaN,naturreservat,skogvern,...,NaN,NaN,NaN,NaN,NaN,110.359650,NaN,NaN,4.913544e+04,NaN
286,VP00001396,http://www.miljodirektoratet.no/naturbase,Korpreiret - utvidelse,20240115000000,https://faktaark.naturbase.no/?id=VP00001396,3412,NaN,NaN,naturreservat,skogvern,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.433789e+05,NaN
287,VP00001397,http://www.miljodirektoratet.no/naturbase,Heggeshuselva - utvidelse,20240115000000,https://faktaark.naturbase.no/?id=VP00001397,3442,NaN,NaN,naturreservat,skogvern,...,NaN,NaN,NaN,332.320496,NaN,NaN,NaN,NaN,6.081541e+04,NaN


### 2. Geometry variables for protected areas

| Protected area | Total area (km2) | Perimeter (km) | Land area (km2) | Perimeter land area (km) |
|----------------|------------|----------------|-----------------|-------------------------|
| *NaturvernId* | *areal_km2* | *omkrets_km* | *landareal_km2* | *landomkrets_km*|
| area A         |    *        |                |                 |                         |

In [29]:
# TODO
# calc terrestrial land area
# difference between bonitet class 17 (hav) and polygon area
#remove_field(db_path, tbl_study_area, "landareal_m2")
#area_difference(db_path, tbl_study_area, "ar50_bon17", "areal_m2", "landareal_m2")

# calc perimeter 
remove_field(db_path, tbl_study_area, "omkrets_m")
geom_peri(db_path, tbl_study_area, "geom", "omkrets_m")

# calc perimeter of terrestrial part of the polygon
remove_field(db_path, tbl_study_area, "landomkrets_m")

# create new table with only terrestrial part of the protected areas
output_tbl = "foreslatt_vern_land"
split_by_polygon(db_path, tbl_study_area, id, tbl_ar50, ar50_field, output_tbl)

# calc peri
# calc area (check what happens with vernID with multiple polygons)


# add column to foreslatt_vern using





An error occurred: Catalog Error: Scalar Function with name st_peri does not exist!
Did you mean "pi"?
LINE 3:                 SET omkrets_m = ST_Peri(geom)
                                        ^


NameError: name 'split_by_polygon' is not defined

***
$\mathbf{\text{Shape Index}}$<br>
***
The shape index is a measure of how compact the shape is compared to a circle with the same area. The shape index is calculated for the entire protected area and the land area. The shape index is a value between 0 and 1, where 0 is a perfect circle and 1 is a long and narrow shape.

- $P$ = perimeter
- $r$ = radius
- $A$ = area


**Shape Index:**&emsp;      $SI = \frac{P}{2\pi r}$

**Radius:**&emsp; $r = \sqrt{\frac{A}{\pi}}$

In [ ]:
# calc formindex

**Bioklimatiske soner**

| Protected area | Sone 1 (%) | Sone 2 (%) | Sone 3 (%) | ... |
|----------------|------------|------------|------------|-----|
| *NaturvernId*  |*bioklima_1*|*bioklima_2*|*bioklima_3*|...  |
| area A         |            |            |            |     |

In [ ]:
# TODO
if run_bioklima:
    tbl_study_area = "foreslatt_vern"
    id = "identifikasjon_lokalId"
    tbl_ar50 = "bioklima"
    ar50_field = "Sone_Kode"
    bioklima_zones = ["6SO-1", "6SO-2", "osv"]
    for area_class in bioklima_zones: # 1-18
        print(f"Calculating Bonitet class {area_class}")
        new_field = f"ar50_bon{area_class}" # ar50_bon1
        remove_field(db_path, tbl_study_area, new_field)
        ar50_area_class(db_path, tbl_study_area, id, tbl_ar50, ar50_field, area_class, new_field)

**Infrastrukturindeks**

| Protected area | Mean value | Lav (%) | Middels (%) | Høy (%) | ... |
|----------------|------------|---------|-------------|---------|-----|
| area A         |            |         |             |         |     |


**Høydelag**

| Protected area | 0-300 m (%) | 301-600 m (%) | 601-900 m (%) | over 900 m (%) |
|----------------|-------------|---------------|---------------|----------------|
| area A         |             |               |               |                |


### 3. Regional Statistics


Administrative regions:
- Municipalities
- Counties
- Regions 
    - **Nord**: Finnmark, Troms, Nordland
    - **Midt**: Trøndelag, Møre og Romsdal
    - **Sør**: Agder, Vestfold, Telemark
    - **Øst**: Østfold, Akershus, Oslo, Innlandet, Buskerud
    - **Vest**: Vestland, Rogaland


| Protected area | Municipality | County | Region |
|----------------|---------|-------|--------|
| *NaturvernId*  |*kommune*|*fylke*|*region*|
| area A         |         |       |        |

*Regional datasett*:

| Region | Area | Protected Area | Land Protected Area | Marine Protected Area |
|--------|------|----------------|---------------------|-----------------------|
| *Region* | *areal* | *a_vern* | *a_landvern* | *a_marinvern* |
| Nord ||||||


###  4. Grid Statistics (10x10 km2)

**Rutenettstatistikk:**

| SSB grid cell | area | Density (land + marine) | Density (land) | 
| --------------|------|-------------------------|----------------|
| *grid_ID* | *grid_areal* | *tetthet_tot_vern* | *tetthet_landvern* | 
| cell 1| 100 km2 | ||





***
$\mathbf{\text{Density}}$<br>
***
The density of protected area per 10x10km (SSB grid) is calculated by dividing the area of the protected area by 100 km^2.

**Density:**&emsp;      $Density = \frac{A}{100 km^2}$



In [ ]:
# code for density calculation here
# check land cover fraction notebook


# Load admin 
# group by region

# calc region area
geom_area()
# Select by region
# SUM protected area in region 
# Area % = SUM(protected_area_m2)/(areal_m2) * 100

